In [1]:
# Import packages
import tensorflow as tf
# tf.enable_eager_execution()

from keras.datasets import imdb
from keras import preprocessing
from keras.models import Sequential
from keras.layers import Flatten, Dense
from keras.layers import Embedding
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras import losses
from keras import metrics
from keras import optimizers
from keras.layers import Dropout

import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

import nltk
# nltk.download('punkt')
# nltk.download('stopwords')

# Downgrate numpy to fix a problem
# !pip install numpy==1.16.2
import numpy as np
print(np.__version__)

import matplotlib.pyplot as plt

import os

import numpy as np

import string

# !pip install ipdb
import ipdb # deb

from gensim.models.keyedvectors import KeyedVectors

# Spliting data
from sklearn.model_selection import train_test_split

from sklearn import metrics # For RUC

from nltk.stem import PorterStemmer

import tensorflow_hub as hub
import pandas as pd
import re
import seaborn as sb

# from google.colab import files

from IPython import display

import logging
logging.getLogger('googleapiclient.discovery_cache').setLevel(logging.ERROR)
from catboost import CatBoostRegressor

Using TensorFlow backend.


1.17.3


*Here We are importing the data*

We have decided to import the train and the test wine data

In [2]:
wine_train=pd.read_csv('Datasets/train.csv')
wine_test=pd.read_csv('Datasets/test.csv')
wine_test.drop(['price', 'index'], axis=1, inplace=True)

In [3]:
wine_train.fillna(value='Missing', inplace=True)
wine_test.fillna(value='Missing', inplace=True)

In [4]:
wine_train.shape

(175000, 14)

In [5]:
wine_test.shape

(83210, 13)

In [766]:
# wine_train.dropna(thresh=1, inplace=True)

In [769]:
# wine_train.dropna(inplace=True)

In [770]:
# wine_train.shape

(15245, 14)

**Head of test data and train data**

In [350]:
wine_test.head()

,index,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery,id
0,41855,US,"Sweet new oak stands out, giving this Cabernet...",3D,95.036469,NaN,California,St. Helena,Napa,NaN,NaN,Salvestrin 2007 3D Cabernet Sauvignon (St. Hel...,CABERNET SAUVIGNON,Salvestrin,0
1,10328,France,Charming raspberry aromas intertwine with scen...,Heluicum,90.966405,NaN,Rhône Valley,Collines Rhôdaniennes,NaN,NaN,NaN,NaN,SYRAH,Les Vins de Vienne,1
2,60094,US,"Ripe blackberry, leather and soy show on the n...",Estate,88.964358,NaN,California,Santa Ynez Valley,Central Coast,Matt Kettmann,@mattkettmann,Brave & Maiden 2013 Estate Cabernet Franc (San...,CABERNET FRANC,Brave & Maiden,2
3,48333,US,White flowers and wild anise give this a crisp...,NaN,89.960356,NaN,California,Sonoma Coast,Sonoma,Virginie Boone,@vboone,Les Voleurs 2013 Chardonnay (Sonoma Coast),CHARDONNAY,Les Voleurs,3
4,14498,US,Pinot Gris has been making inroads in Washingt...,NaN,88.075501,NaN,Washington,Columbia Valley (WA),Columbia Valley,Sean P. Sullivan,@wawinereport,Chateau Ste. Michelle 2015 Pinot Gris (Columbi...,PINOT GRIS,Chateau Ste. Michelle,4


In [6]:
wine_train.head()

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery,id
0,Portugal,This is a fine rich balanced wine. It has ripe...,Vila Santa Reserva,88.870874,20.0,Alentejano,NaN,NaN,NaN,NaN,NaN,PORTUGUESE RED,J. Portugal Ramos,32027
1,France,"A solid, chunky wine, with a structure that is...",NaN,88.041695,28.0,Bordeaux,Lalande de Pomerol,NaN,NaN,NaN,NaN,BORDEAUX-STYLE RED BLEND,Château Tour Grand Colombier,71079
2,France,"This is powerful and concentrated, with the hi...",NaN,94.085021,130.0,Bordeaux,Saint-Émilion,NaN,NaN,NaN,NaN,BORDEAUX-STYLE RED BLEND,Château Figeac,32440
3,US,"Rich, ripe and oaky, this Petite Sirah charms ...",Thompson Vineyard,89.869797,34.0,California,Santa Barbara County,Central Coast,NaN,NaN,Jaffurs 2010 Thompson Vineyard Petite Sirah (S...,PETITE SIRAH,Jaffurs,124405
4,US,This wine is a unique in the state blend and f...,McKinley Springs Vineyard,89.017651,24.0,Washington,Horse Heaven Hills,Columbia Valley,Sean P. Sullivan,@wawinereport,Syncline 2016 McKinley Springs Vineyard Rosé (...,ROSé,Syncline,33649


**Data train info**

In [7]:
wine_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175000 entries, 0 to 174999
Data columns (total 14 columns):
country                  174953 non-null object
description              175000 non-null object
designation              122734 non-null object
points                   175000 non-null float64
price                    175000 non-null float64
province                 174953 non-null object
region_1                 146466 non-null object
region_2                 75394 non-null object
taster_name              65509 non-null object
taster_twitter_handle    62190 non-null object
title                    82189 non-null object
variety                  174999 non-null object
winery                   175000 non-null object
id                       175000 non-null int64
dtypes: float64(2), int64(1), object(11)
memory usage: 18.7+ MB


Data Description

In [ ]:
wine_train.describe()

**Install Package to use for**

In [ ]:
# !pip install catboost

The Model that we are going to implement is the cat Boost regression Model

In [771]:
categorical_train = wine_train[wine_train.columns[wine_train.dtypes == 'object']]
categorical_train.head()

,country,description,designation,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
4,US,This wine is a unique in the state blend and f...,McKinley Springs Vineyard,Washington,Horse Heaven Hills,Columbia Valley,Sean P. Sullivan,@wawinereport,Syncline 2016 McKinley Springs Vineyard Rosé (...,ROSé,Syncline
14,US,Vigorous bubbles present a Pinot Noir-like nos...,Sierra Madre Vineyard Crémant,California,Santa Maria Valley,Central Coast,Matt Kettmann,@mattkettmann,Goat Bubbles 2011 Sierra Madre Vineyard Créman...,SPARKLING BLEND,Goat Bubbles
19,US,The delicate nose on this reserve Chard offers...,Estate Bottled Reserve,New York,Finger Lakes,Finger Lakes,Susan Kostrzewa,@suskostrzewa,Lamoreaux Landing 2007 Estate Bottled Reserve ...,CHARDONNAY,Lamoreaux Landing
48,US,"This wine bears the blue label, and includes 2...",Blue Label,Oregon,Applegate Valley,Southern Oregon,Paul Gregutt,@paulgwine,Troon 2014 Blue Label Vermentino (Applegate Va...,VERMENTINO,Troon
53,US,"Merlot is the principal grape here, with Caber...",Cuvée Estelle,Washington,Columbia Valley (WA),Columbia Valley,Paul Gregutt,@paulgwine,Lauren Ashton Cellars 2011 Cuvée Estelle Red (...,BORDEAUX-STYLE RED BLEND,Lauren Ashton Cellars


In [1067]:
wine_train.nunique()

country                      46
description              123811
designation               37931
points                   175000
price                       387
province                    469
region_1                   1279
region_2                     19
taster_name                  20
taster_twitter_handle        16
title                     77412
variety                     707
winery                    16968
id                       124675
dtype: int64

In [772]:
categorical_train.nunique()

country                      1
description              14358
designation               7448
province                     4
region_1                   168
region_2                    17
taster_name                 11
taster_twitter_handle       11
title                    14323
variety                    167
winery                    2551
dtype: int64

**Code to identify the missing values in our train dataset**

In [773]:
wine_train.isnull().sum()

country                  0
description              0
designation              0
points                   0
price                    0
province                 0
region_1                 0
region_2                 0
taster_name              0
taster_twitter_handle    0
title                    0
variety                  0
winery                   0
id                       0
dtype: int64

In [11]:
wine_train.head()

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery,id
0,Portugal,This is a fine rich balanced wine. It has ripe...,Vila Santa Reserva,88.870874,20.0,Alentejano,NaN,NaN,NaN,NaN,NaN,PORTUGUESE RED,J. Portugal Ramos,32027
1,France,"A solid, chunky wine, with a structure that is...",NaN,88.041695,28.0,Bordeaux,Lalande de Pomerol,NaN,NaN,NaN,NaN,BORDEAUX-STYLE RED BLEND,Château Tour Grand Colombier,71079
2,France,"This is powerful and concentrated, with the hi...",NaN,94.085021,130.0,Bordeaux,Saint-Émilion,NaN,NaN,NaN,NaN,BORDEAUX-STYLE RED BLEND,Château Figeac,32440
3,US,"Rich, ripe and oaky, this Petite Sirah charms ...",Thompson Vineyard,89.869797,34.0,California,Santa Barbara County,Central Coast,NaN,NaN,Jaffurs 2010 Thompson Vineyard Petite Sirah (S...,PETITE SIRAH,Jaffurs,124405
4,US,This wine is a unique in the state blend and f...,McKinley Springs Vineyard,89.017651,24.0,Washington,Horse Heaven Hills,Columbia Valley,Sean P. Sullivan,@wawinereport,Syncline 2016 McKinley Springs Vineyard Rosé (...,ROSé,Syncline,33649


In [507]:
# add a column of word counts to both the training and test set
wine_train['desc_len'] = wine_train['description'].apply(lambda x: len(x))
wine_test['desc_len'] = wine_test['description'].apply(lambda x: len(x))

In [130]:
wine_test.head()

,index,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery,id,desc_len
0,41855,US,"Sweet new oak stands out, giving this Cabernet...",3D,95.036469,NaN,California,St. Helena,Napa,NaN,NaN,Salvestrin 2007 3D Cabernet Sauvignon (St. Hel...,CABERNET SAUVIGNON,Salvestrin,0,342
1,10328,France,Charming raspberry aromas intertwine with scen...,Heluicum,90.966405,NaN,Rhône Valley,Collines Rhôdaniennes,NaN,NaN,NaN,NaN,SYRAH,Les Vins de Vienne,1,227
2,60094,US,"Ripe blackberry, leather and soy show on the n...",Estate,88.964358,NaN,California,Santa Ynez Valley,Central Coast,Matt Kettmann,@mattkettmann,Brave & Maiden 2013 Estate Cabernet Franc (San...,CABERNET FRANC,Brave & Maiden,2,300
3,48333,US,White flowers and wild anise give this a crisp...,NaN,89.960356,NaN,California,Sonoma Coast,Sonoma,Virginie Boone,@vboone,Les Voleurs 2013 Chardonnay (Sonoma Coast),CHARDONNAY,Les Voleurs,3,196
4,14498,US,Pinot Gris has been making inroads in Washingt...,NaN,88.075501,NaN,Washington,Columbia Valley (WA),Columbia Valley,Sean P. Sullivan,@wawinereport,Chateau Ste. Michelle 2015 Pinot Gris (Columbi...,PINOT GRIS,Chateau Ste. Michelle,4,201


In [12]:
wine_train['region_1'].apply(lambda x: len(x) ).mean()

12.491097142857143

In [1020]:
description_tr = wine_train.description
description_tst = wine_test.description

# texts_tr = clean_doc(train_df.sentence)
# texts_tst = clean_doc(test_df.sentence)

# Training 92916
# Considers only the top 
# 20,000 words in the dataset
max_words = 250

tokenizer_tr = Tokenizer(num_words=max_words)
# tokenizer_tr = Tokenizer()
tokenizer_tr.fit_on_texts(description_tr)
# data_tr = tokenizer_tr.texts_to_sequences(description_tr)
tfidf_train = tokenizer_tr.texts_to_matrix(description_tr, mode='tfidf')
# bin_tr = tokenizer_tr.texts_to_matrix(description_tr, mode='binary')


# # Testing 
tokenizer_tst = Tokenizer(num_words=max_words)
# # tokenizer_tst = Tokenizer()
tokenizer_tst.fit_on_texts(description_tst)
# data_tst = tokenizer_tst.texts_to_sequences(texts_tst)
tfidf_tst = tokenizer_tr.texts_to_matrix(description_tst, mode='tfidf')
# bin_tst = tokenizer_tr.texts_to_matrix(description_tst, mode='binary')


desc_tdidf_tr = pd.DataFrame(tfidf_train)
desc_tdidf_tst = pd.DataFrame(tfidf_tst)
desc_tdidf_tr = desc_tdidf_tr.add_prefix('desc_')
desc_tdidf_tst = desc_tdidf_tst.add_prefix('desc_')


# desc_tdidf_tr = pd.DataFrame(bin_tr)
# desc_tdidf_tst = pd.DataFrame(bin_tst)

In [1021]:
designation_tr = wine_train.designation
designation_tst = wine_test.designation

# texts_tr = clean_doc(train_df.sentence)
# texts_tst = clean_doc(test_df.sentence)

# Training 92916
# Considers only the top 
# 20,000 words in the dataset
max_words = 20

tokenizer_tr = Tokenizer(num_words=max_words)
# tokenizer_tr = Tokenizer()
tokenizer_tr.fit_on_texts(designation_tr)
# data_tr = tokenizer_tr.texts_to_sequences(description_tr)
tfidf_train = tokenizer_tr.texts_to_matrix(designation_tr, mode='tfidf')
# bin_tr = tokenizer_tr.texts_to_matrix(title_tr, mode='binary')


# # Testing 
tokenizer_tst = Tokenizer(num_words=max_words)
# # tokenizer_tst = Tokenizer()
tokenizer_tst.fit_on_texts(designation_tst)
# data_tst = tokenizer_tst.texts_to_sequences(texts_tst)
tfidf_tst = tokenizer_tr.texts_to_matrix(designation_tst, mode='tfidf')
# bin_tst = tokenizer_tr.texts_to_matrix(title_tst, mode='binary')


designation_tdidf_tr = pd.DataFrame(tfidf_train)
designation_tdidf_tst = pd.DataFrame(tfidf_tst)
designation_tdidf_tr = designation_tdidf_tr.add_prefix('designation_')
designation_tdidf_tst = designation_tdidf_tst.add_prefix('designation_')
# desc_tdidf_tr = pd.DataFrame(bin_tr)
# desc_tdidf_tst = pd.DataFrame(bin_tst)

In [1022]:
province_tr = wine_train.province
province_tst = wine_test.province

# texts_tr = clean_doc(train_df.sentence)
# texts_tst = clean_doc(test_df.sentence)

# Training 92916
# Considers only the top 
# 20,000 words in the dataset
max_words = 15

tokenizer_tr = Tokenizer(num_words=max_words)
# tokenizer_tr = Tokenizer()
tokenizer_tr.fit_on_texts(province_tr)
# data_tr = tokenizer_tr.texts_to_sequences(description_tr)
tfidf_train = tokenizer_tr.texts_to_matrix(province_tr, mode='tfidf')
# bin_tr = tokenizer_tr.texts_to_matrix(title_tr, mode='binary')


# # Testing 
tokenizer_tst = Tokenizer(num_words=max_words)
# # tokenizer_tst = Tokenizer()
tokenizer_tst.fit_on_texts(province_tst)
# data_tst = tokenizer_tst.texts_to_sequences(texts_tst)
tfidf_tst = tokenizer_tr.texts_to_matrix(province_tst, mode='tfidf')
# bin_tst = tokenizer_tr.texts_to_matrix(title_tst, mode='binary')


province_tdidf_tr = pd.DataFrame(tfidf_train)
province_tdidf_tst = pd.DataFrame(tfidf_tst)
province_tdidf_tr = province_tdidf_tr.add_prefix('province_')
province_tdidf_tst = province_tdidf_tst.add_prefix('province_')
# desc_tdidf_tr = pd.DataFrame(bin_tr)
# desc_tdidf_tst = pd.DataFrame(bin_tst)

In [1023]:
region_1_tr = wine_train.region_1
region_1_tst = wine_test.region_1

# texts_tr = clean_doc(train_df.sentence)
# texts_tst = clean_doc(test_df.sentence)

# Training 92916
# Considers only the top 
# 20,000 words in the dataset
max_words = 15

tokenizer_tr = Tokenizer(num_words=max_words)
# tokenizer_tr = Tokenizer()
tokenizer_tr.fit_on_texts(region_1_tr)
# data_tr = tokenizer_tr.texts_to_sequences(description_tr)
tfidf_train = tokenizer_tr.texts_to_matrix(region_1_tr, mode='tfidf')
# bin_tr = tokenizer_tr.texts_to_matrix(title_tr, mode='binary')


# # Testing 
tokenizer_tst = Tokenizer(num_words=max_words)
# # tokenizer_tst = Tokenizer()
tokenizer_tst.fit_on_texts(region_1_tst)
# data_tst = tokenizer_tst.texts_to_sequences(texts_tst)
tfidf_tst = tokenizer_tr.texts_to_matrix(region_1_tst, mode='tfidf')
# bin_tst = tokenizer_tr.texts_to_matrix(title_tst, mode='binary')


region_1_tdidf_tr = pd.DataFrame(tfidf_train)
region_1_tdidf_tst = pd.DataFrame(tfidf_tst)
region_1_tdidf_tr = region_1_tdidf_tr.add_prefix('region_1_')
region_1_tdidf_tst = region_1_tdidf_tst.add_prefix('region_1_')
# desc_tdidf_tr = pd.DataFrame(bin_tr)
# desc_tdidf_tst = pd.DataFrame(bin_tst)

In [1024]:
title_tr = wine_train.title
title_tst = wine_test.title

# texts_tr = clean_doc(train_df.sentence)
# texts_tst = clean_doc(test_df.sentence)

# Training 92916
# Considers only the top 
# 20,000 words in the dataset
max_words = 30

tokenizer_tr = Tokenizer(num_words=max_words)
# tokenizer_tr = Tokenizer()
tokenizer_tr.fit_on_texts(title_tr)
# data_tr = tokenizer_tr.texts_to_sequences(description_tr)
tfidf_train = tokenizer_tr.texts_to_matrix(title_tr, mode='tfidf')
# bin_tr = tokenizer_tr.texts_to_matrix(title_tr, mode='binary')


# # Testing 
tokenizer_tst = Tokenizer(num_words=max_words)
# # tokenizer_tst = Tokenizer()
tokenizer_tst.fit_on_texts(title_tst)
# data_tst = tokenizer_tst.texts_to_sequences(texts_tst)
tfidf_tst = tokenizer_tr.texts_to_matrix(title_tst, mode='tfidf')
# bin_tst = tokenizer_tr.texts_to_matrix(title_tst, mode='binary')


title_tdidf_tr = pd.DataFrame(tfidf_train)
title_tdidf_tst = pd.DataFrame(tfidf_tst)
title_tdidf_tr = title_tdidf_tr.add_prefix('title_')
title_tdidf_tst = title_tdidf_tst.add_prefix('title_')
# desc_tdidf_tr = pd.DataFrame(bin_tr)
# desc_tdidf_tst = pd.DataFrame(bin_tst)

In [1025]:
variety_tr = wine_train.variety 
variety_tst = wine_test.variety 

# texts_tr = clean_doc(train_df.sentence)
# texts_tst = clean_doc(test_df.sentence)

# Training 92916
# Considers only the top 
# 20,000 words in the dataset
max_words = 15

tokenizer_tr = Tokenizer(num_words=max_words)
# tokenizer_tr = Tokenizer()
tokenizer_tr.fit_on_texts(variety_tr)
# data_tr = tokenizer_tr.texts_to_sequences(description_tr)
tfidf_train = tokenizer_tr.texts_to_matrix(variety_tr, mode='tfidf')
# bin_tr = tokenizer_tr.texts_to_matrix(title_tr, mode='binary')


# # Testing 
tokenizer_tst = Tokenizer(num_words=max_words)
# # tokenizer_tst = Tokenizer()
tokenizer_tst.fit_on_texts(variety_tst)
# data_tst = tokenizer_tst.texts_to_sequences(texts_tst)
tfidf_tst = tokenizer_tr.texts_to_matrix(variety_tst, mode='tfidf')
# bin_tst = tokenizer_tr.texts_to_matrix(title_tst, mode='binary')


variety_tdidf_tr = pd.DataFrame(tfidf_train)
variety_tdidf_tst = pd.DataFrame(tfidf_tst)
variety_tdidf_tr = variety_tdidf_tr.add_prefix('variety_')
variety_tdidf_tst = variety_tdidf_tst.add_prefix('variety_')
# desc_tdidf_tr = pd.DataFrame(bin_tr)
# desc_tdidf_tst = pd.DataFrame(bin_tst)

In [1026]:
winery_tr = wine_train.winery
winery_tst = wine_test.winery

# texts_tr = clean_doc(train_df.sentence)
# texts_tst = clean_doc(test_df.sentence)

# Training 92916
# Considers only the top 
# 20,000 words in the dataset
max_words = 15

tokenizer_tr = Tokenizer(num_words=max_words)
# tokenizer_tr = Tokenizer()
tokenizer_tr.fit_on_texts(winery_tr)
# data_tr = tokenizer_tr.texts_to_sequences(description_tr)
tfidf_train = tokenizer_tr.texts_to_matrix(winery_tr, mode='tfidf')
bin_tr = tokenizer_tr.texts_to_matrix(winery_tr, mode='binary')


# # Testing 
tokenizer_tst = Tokenizer(num_words=max_words)
# # tokenizer_tst = Tokenizer()
tokenizer_tst.fit_on_texts(winery_tst)
# data_tst = tokenizer_tst.texts_to_sequences(texts_tst)
tfidf_tst = tokenizer_tr.texts_to_matrix(winery_tst, mode='tfidf')
bin_tst = tokenizer_tr.texts_to_matrix(winery_tst, mode='binary')


winery_tdidf_tr = pd.DataFrame(tfidf_train)
winery_tdidf_tst = pd.DataFrame(tfidf_tst)
winery_tdidf_tr = winery_tdidf_tr.add_prefix('winery_')
winery_tdidf_tst = winery_tdidf_tst.add_prefix('winery_')
# desc_tdidf_tr = pd.DataFrame(bin_tr)
# desc_tdidf_tst = pd.DataFrame(bin_tst)

In [967]:
# wine_train['price_log'] = np.log(wine_train.price+1)

In [968]:
train_selected = pd.concat([wine_train['points'], wine_train['price'], 
                            desc_tdidf_tr, designation_tdidf_tr, province_tdidf_tr,
                            region_1_tdidf_tr, title_tdidf_tr, variety_tdidf_tr,
                            winery_tdidf_tr], axis=1)
test_selected = pd.concat([wine_test['points'],
                           desc_tdidf_tst, designation_tdidf_tst, province_tdidf_tst,
                           region_1_tdidf_tst, title_tdidf_tst, variety_tdidf_tst,
                           winery_tdidf_tst], axis=1)
# test_selected.drop('price')

In [1027]:
test_selected.head()

,points,desc_0,desc_1,desc_2,desc_3,desc_4,desc_5,desc_6,desc_7,desc_8,...,taster_twitter_handle_@mattkettmann,taster_twitter_handle_@paulgwine,taster_twitter_handle_@suskostrzewa,taster_twitter_handle_@vboone,taster_twitter_handle_@vossroger,taster_twitter_handle_@wawinereport,taster_twitter_handle_@wineschach,taster_twitter_handle_@winewchristina,taster_twitter_handle_@worldwineguys,taster_twitter_handle_Missing
0,95.036469,0.0,1.191034,1.698753,1.377136,0.000000,1.526322,0.8952,1.791127,1.140839,...,0,0,0,0,0,0,0,0,0,1
1,90.966405,0.0,1.476256,0.809465,0.813358,1.430741,1.526322,0.8952,0.000000,0.000000,...,0,0,0,0,0,0,0,0,0,1
2,88.964358,0.0,1.476256,2.112248,1.706924,2.016463,0.000000,0.8952,1.057868,0.000000,...,1,0,0,0,0,0,0,0,0,0
3,89.960356,0.0,1.476256,1.370543,0.813358,0.000000,0.000000,0.8952,0.000000,0.000000,...,0,0,0,1,0,0,0,0,0,0
4,88.075501,0.0,1.191034,1.370543,0.000000,1.430741,0.000000,0.8952,0.000000,0.000000,...,0,0,0,0,0,1,0,0,0,0


In [235]:
def cut_levels(x, threshold, new_value):
    value_counts = x.value_counts()
    labels = value_counts.index[value_counts < threshold]
    x[np.in1d(x, labels)] = new_value

In [136]:
wine_train.country.value_counts()

US                        79129
Italy                     24194
France                    22060
Spain                      9993
Chile                      6893
Argentina                  6323
Portugal                   6066
Australia                  4847
Austria                    3508
New Zealand                3039
Germany                    2950
South Africa               2405
Greece                      903
Israel                      727
Canada                      308
Hungary                     255
Romania                     179
Bulgaria                    146
Uruguay                     124
Slovenia                    116
Turkey                      101
Croatia                      98
Mexico                       86
Georgia                      85
Moldova                      82
Brazil                       58
England                      54
Lebanon                      50
Morocco                      30
Cyprus                       29
Macedonia                    18
Serbia  

In [119]:
# cut_levels(wine_train.country, 30, 'other')
# cut_levels(wine_test.country, 30, 'other')

/home/aims/anaconda3/envs/aims/lib/python3.5/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [1037]:
dummy_fields = ['region_2', 'taster_name', 'taster_twitter_handle']
# train[train.columns[train.dtypes == 'object']], 'country'
# train_select = numerical_train
for each in dummy_fields:
    dummies = pd.get_dummies(wine_train[each], prefix=each)
    train_selected = pd.concat([train_selected, dummies], axis=1)
    dummies = pd.get_dummies(wine_test[each], prefix=each)
    test_selected = pd.concat([test_selected, dummies], axis=1)

In [798]:
wine_train.country.value_counts()

US                        79129
Italy                     24194
France                    22060
Spain                      9993
Chile                      6893
Argentina                  6323
Portugal                   6066
Australia                  4847
Austria                    3508
New Zealand                3039
Germany                    2950
South Africa               2405
Greece                      903
Israel                      727
Canada                      308
Hungary                     255
Romania                     179
Bulgaria                    146
Uruguay                     124
Slovenia                    116
Turkey                      101
Croatia                      98
Mexico                       86
Georgia                      85
Moldova                      82
Brazil                       58
England                      54
Lebanon                      50
Missing                      47
Morocco                      30
Cyprus                       29
Macedoni

In [921]:
test_selected.head()

,points,desc_0,desc_1,desc_2,desc_3,desc_4,desc_5,desc_6,desc_7,desc_8,...,taster_twitter_handle_@laurbuzz,taster_twitter_handle_@mattkettmann,taster_twitter_handle_@paulgwine,taster_twitter_handle_@suskostrzewa,taster_twitter_handle_@vboone,taster_twitter_handle_@vossroger,taster_twitter_handle_@wawinereport,taster_twitter_handle_@wineschach,taster_twitter_handle_@winewchristina,taster_twitter_handle_@worldwineguys
0,95.036469,0.0,1.191034,1.698753,1.377136,0.000000,1.526322,0.8952,1.791127,1.140839,...,0,0,0,0,0,0,0,0,0,0
1,90.966405,0.0,1.476256,0.809465,0.813358,1.430741,1.526322,0.8952,0.000000,0.000000,...,0,0,0,0,0,0,0,0,0,0
2,88.964358,0.0,1.476256,2.112248,1.706924,2.016463,0.000000,0.8952,1.057868,0.000000,...,0,1,0,0,0,0,0,0,0,0
3,89.960356,0.0,1.476256,1.370543,0.813358,0.000000,0.000000,0.8952,0.000000,0.000000,...,0,0,0,0,1,0,0,0,0,0
4,88.075501,0.0,1.191034,1.370543,0.000000,1.430741,0.000000,0.8952,0.000000,0.000000,...,0,0,0,0,0,0,1,0,0,0


In [922]:
train_selected.head()

,points,price,desc_0,desc_1,desc_2,desc_3,desc_4,desc_5,desc_6,desc_7,...,taster_twitter_handle_@laurbuzz,taster_twitter_handle_@mattkettmann,taster_twitter_handle_@paulgwine,taster_twitter_handle_@suskostrzewa,taster_twitter_handle_@vboone,taster_twitter_handle_@vossroger,taster_twitter_handle_@wawinereport,taster_twitter_handle_@wineschach,taster_twitter_handle_@winewchristina,taster_twitter_handle_@worldwineguys
0,88.870874,20.0,0.0,0.703444,1.370543,1.377136,0.845019,0.901471,0.8952,1.057868,...,0,0,0,0,0,0,0,0,0,0
1,88.041695,28.0,0.0,1.476256,1.698753,1.706924,0.845019,1.526322,0.0000,1.057868,...,0,0,0,0,0,0,0,0,0,0
2,94.085021,130.0,0.0,1.191034,1.370543,0.000000,0.845019,1.526322,0.8952,1.791127,...,0,0,0,0,0,0,0,0,0,0
3,89.869797,34.0,0.0,1.191034,1.698753,0.000000,0.000000,0.901471,0.8952,1.057868,...,0,0,0,0,0,0,0,0,0,0
4,89.017651,24.0,0.0,1.678624,1.698753,1.706924,0.845019,1.526322,0.8952,1.057868,...,0,0,0,0,0,0,1,0,0,0


In [878]:
train_selected.head()

,points,price,desc_0,desc_1,desc_2,desc_3,desc_4,desc_5,desc_6,desc_7,...,taster_twitter_handle_@mattkettmann,taster_twitter_handle_@paulgwine,taster_twitter_handle_@suskostrzewa,taster_twitter_handle_@vboone,taster_twitter_handle_@vossroger,taster_twitter_handle_@wawinereport,taster_twitter_handle_@wineschach,taster_twitter_handle_@winewchristina,taster_twitter_handle_@worldwineguys,taster_twitter_handle_Missing
0,88.870874,20.0,0.0,0.703444,1.370543,1.377136,0.845019,0.901471,0.8952,1.057868,...,0,0,0,0,0,0,0,0,0,1
1,88.041695,28.0,0.0,1.476256,1.698753,1.706924,0.845019,1.526322,0.0000,1.057868,...,0,0,0,0,0,0,0,0,0,1
2,94.085021,130.0,0.0,1.191034,1.370543,0.000000,0.845019,1.526322,0.8952,1.791127,...,0,0,0,0,0,0,0,0,0,1
3,89.869797,34.0,0.0,1.191034,1.698753,0.000000,0.000000,0.901471,0.8952,1.057868,...,0,0,0,0,0,0,0,0,0,1
4,89.017651,24.0,0.0,1.678624,1.698753,1.706924,0.845019,1.526322,0.8952,1.057868,...,0,0,0,0,0,1,0,0,0,0


In [879]:
train_selected.shape

(175000, 287)

In [880]:
test_selected.head()

,points,desc_0,desc_1,desc_2,desc_3,desc_4,desc_5,desc_6,desc_7,desc_8,...,taster_twitter_handle_@mattkettmann,taster_twitter_handle_@paulgwine,taster_twitter_handle_@suskostrzewa,taster_twitter_handle_@vboone,taster_twitter_handle_@vossroger,taster_twitter_handle_@wawinereport,taster_twitter_handle_@wineschach,taster_twitter_handle_@winewchristina,taster_twitter_handle_@worldwineguys,taster_twitter_handle_Missing
0,95.036469,0.0,1.191034,1.698753,1.377136,0.000000,1.526322,0.8952,1.791127,1.140839,...,0,0,0,0,0,0,0,0,0,1
1,90.966405,0.0,1.476256,0.809465,0.813358,1.430741,1.526322,0.8952,0.000000,0.000000,...,0,0,0,0,0,0,0,0,0,1
2,88.964358,0.0,1.476256,2.112248,1.706924,2.016463,0.000000,0.8952,1.057868,0.000000,...,1,0,0,0,0,0,0,0,0,0
3,89.960356,0.0,1.476256,1.370543,0.813358,0.000000,0.000000,0.8952,0.000000,0.000000,...,0,0,0,1,0,0,0,0,0,0
4,88.075501,0.0,1.191034,1.370543,0.000000,1.430741,0.000000,0.8952,0.000000,0.000000,...,0,0,0,0,0,1,0,0,0,0


In [749]:
# test_selected.drop(['country_US-France'],axis=1, inplace=True)

In [714]:
test_selected.shape

(83210, 218)

In [571]:
# convert categorical columns to integers
category_cols = train_selected.columns[train_selected.dtypes == 'object']
for header in category_cols:
    train_selected[header] = train_selected[header].astype('category').cat.codes
#     wine_test[header] = wine_test[header].astype('category').cat.codes

In [572]:
# wine_train['country']=pd.Categorical(wine_train['country'])
# wine_train["country"] = wine_train.country.cat.codes

# wine_train['description']=pd.Categorical(wine_train['description'])
# wine_train["description"] = wine_train.description.cat.codes

# wine_train['designation']=pd.Categorical(wine_train['designation'])
# wine_train["designation"] = wine_train.designation.cat.codes

# wine_train['province']=pd.Categorical(wine_train['province'])
# wine_train["province"] = wine_train.province.cat.codes

# wine_train['region_1']=pd.Categorical(wine_train['region_1'])
# wine_train["region_1"] = wine_train.region_1.cat.codes

# wine_train['region_2']=pd.Categorical(wine_train['region_2'])
# wine_train["region_2"] = wine_train.region_2.cat.codes


# wine_train['taster_name']=pd.Categorical(wine_train['taster_name'])
# wine_train["taster_name"] = wine_train.taster_name.cat.codes

# wine_train['taster_twitter_handle']=pd.Categorical(wine_train['taster_twitter_handle'])
# wine_train["taster_twitter_handle"] = wine_train.taster_twitter_handle.cat.codes

# wine_train['title']=pd.Categorical(wine_train['title'])
# wine_train["title"] = wine_train.title.cat.codes

# wine_train['variety']=pd.Categorical(wine_train['variety'])
# wine_train["variety"] = wine_train.variety.cat.codes

# wine_train['winery']=pd.Categorical(wine_train['winery'])
# wine_train["winery"] = wine_train.winery.cat.codes

In [573]:
# wine_train['price_log']=np.log(wine_train.price+1)

Let us declare our model variable and the corresponding features

In [988]:
train_selected.head()

,points,price,desc_0,desc_1,desc_2,desc_3,desc_4,desc_5,desc_6,desc_7,...,taster_twitter_handle_@mattkettmann,taster_twitter_handle_@paulgwine,taster_twitter_handle_@suskostrzewa,taster_twitter_handle_@vboone,taster_twitter_handle_@vossroger,taster_twitter_handle_@wawinereport,taster_twitter_handle_@wineschach,taster_twitter_handle_@winewchristina,taster_twitter_handle_@worldwineguys,taster_twitter_handle_Missing
0,88.870874,20.0,0.0,0.703444,1.370543,1.377136,0.845019,0.901471,0.8952,1.057868,...,0,0,0,0,0,0,0,0,0,1
1,88.041695,28.0,0.0,1.476256,1.698753,1.706924,0.845019,1.526322,0.0000,1.057868,...,0,0,0,0,0,0,0,0,0,1
2,94.085021,130.0,0.0,1.191034,1.370543,0.000000,0.845019,1.526322,0.8952,1.791127,...,0,0,0,0,0,0,0,0,0,1
3,89.869797,34.0,0.0,1.191034,1.698753,0.000000,0.000000,0.901471,0.8952,1.057868,...,0,0,0,0,0,0,0,0,0,1
4,89.017651,24.0,0.0,1.678624,1.698753,1.706924,0.845019,1.526322,0.8952,1.057868,...,0,0,0,0,0,1,0,0,0,0


In [1028]:
# y = wine_train['price_log']
y = train_selected['price']

# X=wine_train[['country', 'description','designation',
#               'province', 'region_1', 'region_2', 'taster_name',
#               'taster_twitter_handle', 'title', 'variety', 'winery']]
    
category_cols = ['designation','points','province'
              ,'country','variety','winery', 'region_1', 'region_2']

# X = wine_train[category_cols]
# X = train_selected.drop([''])
X = train_selected.drop(['price'], axis=1)
# wine_test_selected = wine_test[category_cols]

In [1029]:
train_selected.shape

(175000, 167)

Let's Train the model

In [1030]:
from sklearn.model_selection import train_test_split

X_train, X_test,y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=55)

In [1031]:
X_train.shape

(140000, 166)

Building the model by using CatBoost package

## Random Forest

In [1032]:
from sklearn.ensemble import RandomForestRegressor

In [1038]:
model = RandomForestRegressor(n_estimators=100, criterion="mse", n_jobs=-1)
# model = RandomForestRegressor()

In [1039]:
model.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
                      oob_score=False, random_state=None, verbose=0,
                      warm_start=False)

In [1040]:
y_preds = model.predict(X_test)
y_preds

array([29.7 , 30.62, 43.67, ..., 64.28, 96.49, 36.55])

In [1041]:
print (np.sqrt(metrics.mean_squared_error(y_preds,y_test)))
# 23.592476287693316

25.07743387065636


## Catboost

In [1042]:
# model_cat = CatBoostRegressor(iterations=100,
#                               learning_rate=0.03)

model_cat = CatBoostRegressor()

In [1043]:
# cat_dims = [X_train.columns.get_loc(i) for i in category_cols[:-1]] 

In [1044]:
# model_cat.fit(X, y, silent=True) #, cat_features=cat_dims

In [1045]:
model_cat.fit(X_train, y_train, silent=True) #, cat_features=cat_dims

In [1046]:
# Total Training 
# model_cat.fit(X, y, silent=True)

Model evaluation

In [1047]:
preds = model_cat.predict(X_test)

print (np.sqrt(metrics.mean_squared_error(preds,y_test)))

# 29.57211208231976 = 35.9633215 (Best)

27.688437973410867


In [1048]:
# >>> data = [[0, 0], [0, 0], [1, 1], [1, 1]]
# >>> scaler = StandardScaler()
# >>> print(scaler.fit(data))
# >>> print(scaler.mean_)
# data_s = scaler.transform(data)
# >>> print(scaler.transform(data))
# >>> print(scaler.inverse_transform(data_s))

In [1055]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train_ = scaler.transform(X_train)
X_test_ = scaler.transform(X_test)

# scaler_y = StandardScaler()
# scaler.fit(y_train.values.reshape(-1,1))
y_train_ = y_train.values.reshape(-1,1)

In [1066]:
model_d = tf.keras.Sequential()
# model.add(Embedding(max_words, embedding_dim, input_length=maxlen))
# model.add(tf.keras.layers.Embedding(max_words, 
#                                     embedding_dim, 
#                                     input_length=x_train.shape[1],
#                                     embeddings_initializer=tf.keras.initializers.Constant(
#                                         embedding_matrix),
#                                     trainable = False    
#                                    ))
# model.add(tf.keras.layers.Flatten())
model_d.add(tf.keras.layers.Dense(512, input_shape=(X_train.shape[1], ), kernel_initializer='normal', activation='relu'))
# model_d.add(tf.keras.layers.Dropout(0.5))
model_d.add(tf.keras.layers.Dense(512, activation='relu'))
model_d.add(tf.keras.layers.Dense(1, kernel_initializer='normal'))
model_d.summary()

class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('val_acc')>0.66):
      print("\nReached 90% val_acc so cancelling training!")
      self.model.stop_training = True
      
callbacks = myCallback()

monitor = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, verbose=1, mode='auto')

# Compile model
model_d.compile(loss='mean_squared_error', optimizer='adam')
# fit network
model_d.fit(X_train_, y_train_, epochs=50,verbose=2, validation_split=0.1)#, 
#           validation_split=0.01, callbacks=[callbacks])

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_36 (Dense)             (None, 512)               85504     
_________________________________________________________________
dense_37 (Dense)             (None, 512)               262656    
_________________________________________________________________
dense_38 (Dense)             (None, 1)                 513       
Total params: 348,673
Trainable params: 348,673
Non-trainable params: 0
_________________________________________________________________
Train on 126000 samples, validate on 14000 samples
Epoch 1/50
126000/126000 - 10s - loss: 1112.8396 - val_loss: 655.5839
Epoch 2/50
126000/126000 - 10s - loss: 868.0044 - val_loss: 613.7515
Epoch 3/50
126000/126000 - 10s - loss: 802.5356 - val_loss: 596.3939
Epoch 4/50
126000/126000 - 10s - loss: 752.0045 - val_loss: 665.2940
Epoch 5/50
126000/126000 - 10s - loss: 702.1416 -

In [1061]:
model_d.evaluate(X_test_, y_test.values.reshape(-1,1), verbose=0)
# print('Test Accuracy: %f' % (acc*100))

782.0206552141462

In [1063]:
print (np.sqrt(metrics.mean_squared_error(model_d.predict(X_test_),y_test.values.reshape(-1,1))))

27.964632169443114


## Grid search

In [ ]:
from paramsearch import paramsearch
from itertools import product,chain
from sklearn.model_selection import KFold

In [ ]:
params = {'depth':[3,1,2,6,4,5,7,8,9,10],
          'iterations':[250,100,500,1000],
          'learning_rate':[0.03,0.001,0.01,0.1,0.2,0.3], 
          'l2_leaf_reg':[3,1,5,10,100],
          'border_count':[32,5,10,20,50,100,200],
          'ctr_border_count':[50,5,10,20,100,200],
          'thread_count':4}

In [ ]:
# this function does 3-fold crossvalidation with catboostclassifier          
def crossvaltest(params,train_set,train_label,cat_dims,n_splits=3):
    kf = KFold(n_splits=n_splits,shuffle=True) 
    res = []
    for train_index, test_index in kf.split(train_set):
        train = train_set.iloc[train_index,:]
        test = train_set.iloc[test_index,:]

        labels = train_label.ix[train_index]
        test_labels = train_label.ix[test_index]

        clf = cb.CatBoostClassifier(**params)
        clf.fit(train, np.ravel(labels), cat_features=cat_dims)

        res.append(np.mean(clf.predict(test)==np.ravel(test_labels)))
    return np.mean(res)

In [ ]:
# this function runs grid search on several parameters
# def catboost_param_tune(params,train_set,train_label,cat_dims=None,n_splits=3):
#     ps = paramsearch(params)
#     # search 'border_count', 'l2_leaf_reg' etc. individually 
#     #   but 'iterations','learning_rate' together
#     for prms in chain(ps.grid_search(['border_count']),
#                       ps.grid_search(['ctr_border_count']),
#                       ps.grid_search(['l2_leaf_reg']),
#                       ps.grid_search(['iterations','learning_rate']),
#                       ps.grid_search(['depth'])):
#         res = crossvaltest(prms,train_set,train_label,cat_dims,n_splits)
#         # save the crossvalidation result so that future iterations can reuse the best parameters
#         ps.register_result(res,prms)
#         print(res,prms,s,'best:',ps.bestscore(),ps.bestparam())
#     return ps.bestparam()

# bestparams = catboost_param_tune(params,X_train,y_train)

# Xboost

In [683]:
import xgboost as xgb

In [1004]:
xgb_model = xgb.XGBRegressor(max_depth=10,
    learning_rate=0.03,
    n_estimators=100)

xgb_model.fit(X_train, y_train)
# xgb_model.fit(X, y)


y_pred_xgb = xgb_model.predict(X_test)

/home/aims/anaconda3/envs/aims/lib/python3.5/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[14:09:37] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [1005]:
print (np.sqrt(metrics.mean_squared_error(y_pred_xgb,y_test)))

27.886551783556822


In [1006]:
y_pred_xgb

array([27.915133, 27.962046, 53.006348, ..., 61.045506, 93.08382 ,
       25.281893], dtype=float32)

# Testing

In [ ]:
# wine_test['country']=pd.Categorical(wine_test['country'])
# wine_test["country"] = wine_test.country.cat.codes

# wine_test['description']=pd.Categorical(wine_test['description'])
# wine_test["description"] = wine_test.description.cat.codes

# wine_test['designation']=pd.Categorical(wine_test['designation'])
# wine_test["designation"] = wine_test.designation.cat.codes

# wine_test['province']=pd.Categorical(wine_test['province'])
# wine_test["province"] = wine_test.province.cat.codes

# wine_test['region_1']=pd.Categorical(wine_test['region_1'])
# wine_test["region_1"] = wine_test.region_1.cat.codes

# wine_test['region_2']=pd.Categorical(wine_test['region_2'])
# wine_test["region_2"] = wine_test.region_2.cat.codes


# wine_test['taster_name']=pd.Categorical(wine_test['taster_name'])
# wine_test["taster_name"] = wine_test.taster_name.cat.codes

# wine_test['taster_twitter_handle']=pd.Categorical(wine_test['taster_twitter_handle'])
# wine_test["taster_twitter_handle"] = wine_test.taster_twitter_handle.cat.codes

# wine_test['title']=pd.Categorical(wine_test['title'])
# wine_test["title"] = wine_test.title.cat.codes

# wine_test['variety']=pd.Categorical(wine_test['variety'])
# wine_test["variety"] = wine_test.variety.cat.codes

# wine_test['winery']=pd.Categorical(wine_test['winery'])
# wine_test["winery"] = wine_test.winery.cat.codes

In [ ]:
# wine_test_selected = wine_test[['description','designation','points','province'
#               ,'country','variety','winery']]

# category_cols = ['description','designation','points','province'
#               ,'country','variety','winery', 'region_1', 'region_2']

# wine_test_selected = wine_test[category_cols]

# wine_test_selected = wine_test[['description','designation','points','province'
#               ,'country','variety','winery', 'region_1', 'region_2']]

In [ ]:
wine_test_selected.columns

In [ ]:
wine_test

In [ ]:
# y = wine_train['price_log']
# y = wine_train['price']

# X=wine_train[['country', 'description','designation',
#               'province', 'region_1', 'region_2', 'taster_name',
#               'taster_twitter_handle', 'title', 'variety', 'winery']]
    
# category_cols = ['description','designation','points','province'
#               ,'country','variety','winery', 'region_1', 'region_2']

# wine_test_selected = wine_test[category_cols]

In [686]:
# preds = model_cat.predict(wine_test_selected)
preds = model.predict(test_selected)

preds

array([207.312,  49.236,  32.944, ...,  34.654,  16.702,  62.126])

In [687]:
wine_test.id

0            0
1            1
2            2
3            3
4            4
         ...  
83205    83205
83206    83206
83207    83207
83208    83208
83209    83209
Name: id, Length: 83210, dtype: int64

In [688]:
type(preds)#.size

numpy.ndarray

In [689]:
preds.reshape(-1,1).shape

(83210, 1)

In [690]:
output = wine_test[['id']]
output['price']=preds.reshape(-1,1)

/home/aims/anaconda3/envs/aims/lib/python3.5/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [691]:
output.head()

,id,price
0,0,207.312
1,1,49.236
2,2,32.944
3,3,30.228
4,4,18.340


In [692]:
output.to_csv('final_tfidf_3.csv',index=False)

In [ ]:
!ls